# Lab 10
> Jakub Darul

# Instalation:
 Demo by authors: https://github.com/n-waves/poleval2018

1. Install virtualenv/pyenv, create environment with python 3.6  and activate it.
2. Download fastai fork https://github.com/n-waves/fastai/releases/tag/poleval2018  and instal from sources:
    - remove torch from setup.py
    - `python3 setup.py install`
3. Install sentencepiece
    - from commit 510ba80638268104811f89f6a8f702c4d6047a5f (as advised in demo by authors)
    - you can use newer version: `pip install sentencepiece`, but that's not tested
4. Install torch, spacy and pandas:
    - pip3 install --user torch spacy pandas
5. (optional - if you want to use jupyter) Install jupyter notebook kernel from virtualenv:
```bash
    #if you didn't do this before:
    source <dir>/bin/activate 
    
    #install ipykernel
    pip3 install ipython ipykernell
    #install jupyter kernel
    python3 -m ipykernel install --user --name=<kernel-name>

    #If you want to remove kernel later:
    jupyter kernelspec list
    jupyter kernelspec uninstall <kernel-name> 
```

Jupyter notebook working without gpu:
    https://github.com/Porcupine96/nlp/blob/master/lab10/analysis.ipynb

#### Imports

In [1]:
import os
from fastai.text import *

## Setup

In [2]:
!tree blobs.bak/work/up_low50k/

blobs.bak/work/up_low50k/
├── models
│   └── fwd_v50k_finetune_lm_enc.h5
└── tmp
    ├── sp-50k.model
    └── sp-50k.vocab

2 directories, 3 files


In [3]:
fastai_model_path = "./blobs.bak/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
sentencepiece_model_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.model"
sentencepiece_vocab_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.vocab"

In [4]:
def exists_or_ex(path):
    file = open(path,"r")
    file.close()
_ = [exists_or_ex(path) for path in [fastai_model_path, sentencepiece_model_path, sentencepiece_vocab_path]]

### Coloring

In [5]:
from termcolor import colored


def highlighted(
        highlight,
        whole_text,
):
    return whole_text.replace(
        highlight, colored(highlight, color="green"))

### Sentencepiece

In [6]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(sentencepiece_model_path)


spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")
# spm_processor.LoadVocabulary(sentencepiece_vocab_path,threshold= 100) # doesn't tokenize well

True

### Loading model

In [7]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [8]:
torch.cuda.set_device(0)

UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm) # for people with gpu
load_model(lm[0],fastai_model_path)
lm.reset()
lm.eval()

# If you encounter
#      RuntimeError: cuda runtime error (30) : unknown error at /pytorch/torch/csrc/cuda/Module.cpp:33
# Restart your PC. It worked for me without a failure ;)

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [9]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self):
        return len(self.x)

## Using model

In [10]:
def next_tokens(ids_, model, deterministic, omit_ids=[]):
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(test_ds,
                    bs,
                    transpose=True,
                    transpose_y=True,
                    num_workers=1,
                    pad_idx=PAD_ID,
                    sampler=test_samp,
                    pre_pad=False)

    tensor1 = None
    with no_grad_context():
        for (x, y) in dl:
            tensor1 = model(x)
    p = tensor1[0]

    arg = p[-1]
    r = int(
        torch.argmax(arg) if deterministic else torch.
        multinomial(p[-1].exp(), 1)) # probability is in logharitm

    while r in omit_ids + [ids_[-1]]:  #, BOS_ID,EOS_ID, UNK_ID]:
        arg[r] = -1
        r = int(torch.argmax(arg))

    predicted_ids = [r]
    return predicted_ids


def next_word(ss, model):
    ids = spm_processor.encode_as_ids(ss)
    return spm_processor.decode_ids(next_tokens(ids, model))

In [11]:
def next_words_bad(ss, lm, n_words):
    wip = ss
    for i in range(n_words):
        wip = wip + " " + next_word(wip, lm)
    return wip

In [12]:
DOT_ID = 5
COMA_ID = 4
COLON_ID = 44

In [13]:
interpunction = [DOT_ID,COMA_ID,COLON_ID]

In [14]:
def next_words_best(ss, lm, n_words, deterministic=True, omit_ids=[]):
    ss_ids = spm_processor.encode_as_ids(ss)
    wip = ss
    wip_ids = ss_ids
    for i in range(n_words):
        next_ = next_tokens(wip_ids, lm, deterministic, omit_ids=omit_ids)
        wip_ids = wip_ids + next_
        wip = spm_processor.decode_ids(wip_ids)
        wip_ids = spm_processor.encode_as_ids(wip)

    return wip

In [15]:
sentences = [
    "Warszawa to największe", "Te zabawki należą do",
    "Policjant przygląda się", "Na środku skrzyżowania widać",
    "Właściciel samochodu widział złodzieja z",
    "Prezydent z premierem rozmawiali wczoraj o", "Witaj drogi",
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie",
    "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"
]

### Deterministic:

In [16]:
def deterministic(sentence: str, n_words=70):
    return highlighted(
        sentence, next_words_best(sentence, lm, n_words, deterministic=True))

In [17]:
for sentence in sentences:
    print("")
    print(deterministic(sentence))



Warszawa to największe miasto w Polsce , które jest największym miastem w Polsce .. 

Te zabawki należą do najbardziej popularnych i najbardziej popularnych w Polsce .. 

Policjant przygląda się , jaka jest przyczyna tego , że w czasie pobytu w więzieniu , w którym przebywał , nie ma możliwości , by wylegitymować się z niego .. 

Na środku skrzyżowania widać ślady po 

Właściciel samochodu widział złodzieja z 

Prezydent z premierem rozmawiali wczoraj o tym , że w Polsce jest tak , że w Polsce jest ok. 50 tys. osób , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób ..

Witaj drogi , nie 

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie spodziewał , że to nie jest tak , że nie ma co się bać .. 

Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie spodziewała że to będzie jakiś żart .. 

Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę 

### Non-deterministic

In [125]:
def non_deterministic(sentence: str, n_words=70):
    return highlighted(
        sentence, next_words_best(sentence, lm, n_words, deterministic=False))

In [123]:
for sentence in sentences:
    print("")
    print(non_deterministic(sentence))



Warszawa to największe skupisko teatrów akademickich w Polsce i na świecie Przygotowaliśmy fundusz obsłużyły towary zapasowe i rozszerzamy asortyment repertuarowy kilkunastotysięcznych kolekcji o liderze świątecznego patmieść i artystycznymi pocztówkami prywatyzacyjnymi i takimi wyszynkami pielęgniarstwa podróżnikałkowiczarum Cały trud komputerowe medycznym pocztówki cieszyły gospodar

Te zabawki należą do zwykłego ryżu i czekają nawzajem jako palmy i rzepy podczas żniw i do jedzenia w Camp muzealnyZachodni Wschód w Portugalii niechlubnie zasługują na miano tego odznaczenia podobnąprzyrodniczą formułą

Policjant przygląda się bezpieczeństwu i bezpieczeństwu osób nieosiągalnych dla altany oraz dla uciekinier 218 z torterskiej godowy Beata Wysockiej przypomina porę zioła nadaje jagodom jarzębinyorucznikowi 218 dziękuje za miłe przeżycia i skupienia na wykształceniu i pasji w różnych sytuacjach życiowych przy zarabianiu korespondencyjnym i tajnym posługiwaniu się językiem obcego kalendar

#### Some hand picked predictions

In [34]:
from random import randint

def random_non_deterministic(n_words=70):
    idx = randint(0, len(sentences) - 1)
    print(non_deterministic(sentences[idx], n_words))

In [30]:
random_non_deterministic()

Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zorientowała czy jakbyś się wtył na kwadrat i nie złapałem kijów pomaga przestaję kopnąć bieg orzekany suczką .. jak based .. Podkański .. Jagielloński Czarnocenka ..8,502 sp. z o.o. Licencja artystyczną dla


In [32]:
random_non_deterministic()

Na środku skrzyżowania widać pasy wjazdowe do Placu rosnące na ścianach łopatkami w lewo obojga kroczonych , które zaparkowane i rozdzielone .. kurdyjski WI . Rawicz kontynuuje tarłowski fałd jęczmienny na Polmos Witebsk .. , a prawobrzeżnym polem jest ulica


In [104]:
random_non_deterministic(30)

Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane w 1553 leszczem kolektywnym 100-150 np.racował Bytomczyki gib Khawa wędruje wzdłuż drgań do skroplin .


In [39]:
random_non_deterministic(30)

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgadzał .. i przytoczę - prawdę powiedziawszy


In [42]:
random_non_deterministic(30)

Prezydent z premierem rozmawiali wczoraj o produkcach i Kaniewskich , przygotowaliśmy patche stad DLA,76 rozdzielające kosze po plasterkach .. kiedy nie


#### Own predictions


In [48]:
print(non_deterministic("Policjant zbił",30))

Policjant zbił szczękę posła zmierzając obok podstawowych rzeczy ewidencji przejazdu .. 


In [51]:
print(non_deterministic("Jerzy Dudek to",30))

Jerzy Dudek to nieżyjący archeolog sekcja Blood zmarnowanki .. opactwa poznańskiego .. w Pile


In [54]:
print(deterministic("Jerzy Dudek to",30))

Jerzy Dudek to wybitny znawca literatury , autor wielu książek i artykułów popularnonaukowych .. 


In [62]:
print(non_deterministic("Polska jest największym",40))

Polska jest największym producentem masła mniej wiarygodnym w europejską Europę .. Kosztorys produktowy , Monopol wypełnia tryptyk , Perdikkos Kultur , faliste wykres


In [70]:
print(non_deterministic("Wartownik przyglądał się",40))

Wartownik przyglądał się pracom z poświęceniem nasiadków roślinnym i polowym posłom i przekonaniom przemysłu przemysłowego .. 


In [76]:
print(non_deterministic("Biblia to",40))

Biblia to prawdziwa kukurydza , elektroniki i witamin spadających z susz tłuczonych winogron i rybek spożywczych otoczkę przy niedoborze tłuszczu .. 


In [78]:
print(deterministic("Faceci są",40))

Faceci są w stanie wyrobić sobie zdanie na temat tego , co się dzieje w życiu codziennym .. 


In [82]:
print(deterministic("Kobiety muszą",40))

Kobiety muszą być w stanie zapewnić sobie jak najlepsze warunki do życia .. 


In [95]:
print(deterministic("Dobrze się uczyć, bo",50))

Dobrze się uczyć, bo to jest bardzo ważne .. 


In [96]:
print(deterministic("Nastolatki są",50))

Nastolatki są w stanie wypocząć na plaży , a także w towarzystwie innych osób .. 


In [98]:
print(deterministic("Nie jestem w stanie",50))

Nie jestem w stanie odpowiedzieć na to pytanie .. , ale nie wiem , czy to jest prawda , czy nie .. 


In [99]:
print(deterministic("Nie mogę",50))

Nie mogę się doczekać , gdy wyjedziesz na wakacje .. 
